<a href="https://colab.research.google.com/github/ak2742/mlplay/blob/Fine-Tuning/08)_gemma_2b_it.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install libraries

!pip install -q -U huggingface_hub
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes

In [ ]:
#@title Log in to HF

from huggingface_hub import login
from google.colab import userdata
access_token = userdata.get('HF_TOKEN')
login(token=access_token)

In [ ]:
#@title Create Model

%%time
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Check what type of Device enabled (GPU or CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
# Load the model
quantization_config = BitsAndBytesConfig(
                                        load_in_4bit=True,
                                        bnb_4bit_use_double_quant=True,
                                        bnb_4bit_quant_type="nf4",
                                        bnb_4bit_compute_dtype=torch.bfloat16,)
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", quantization_config=quantization_config, low_cpu_mem_usage=True)# Use the model

In [ ]:
#@title Generate basic response

%%time
input_text = "What is the best thing about Kaggle?"
# Encode input text to PyTorch tensors
input_ids = tokenizer(input_text, return_tensors="pt").to(device)

outputs = model.generate(**input_ids, do_sample=True, max_new_tokens=100, temperature=0.5)

print(tokenizer.decode(outputs[0]))

In [ ]:
#@title load CSV

import pandas as pd
data = pd.read_csv('/content/sample_data/california_housing_test.csv')
data.head(2)
# data['latitude'][5]

In [ ]:
#@title generate summary

def generate_summary(ctx):
    prompt_template = f"""Provide summary of following context in 200 words.

    Provide only useful information:

    Context: {ctx}"""


    messages = [
        {"role": "user", "content": prompt_template},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    # print(prompt, "\n-------------------x-----------------------")
    input_ids = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt").to('cuda')

    outputs = model.generate(input_ids, max_new_tokens=600)

    response = tokenizer.decode(outputs[0])
    return response

In [ ]:
#@title Regex funtion to post-process the output.

import re
def extract_content(text):
    # Find the index of '<start_of_turn>model'
    index = text.find('<start_of_turn>model')

    # Extract the content after '<start_of_turn>model'
    if index != -1:
        content_after_model = text[index + len('<start_of_turn>model'):-5].strip()
    else:
        return "Content not found after '<start_of_turn>model'"
    return content_after_model

In [ ]:
ctx = """It was a Thursday, but it felt like a Monday to John. And John loved Mondays. He thrived at work. He dismissed the old cliché of dreading Monday mornings and refused to engage in water-cooler complaints about “the grind” and empty conversations that included the familiar parry “How was your weekend?” “Too short!”. Yes, John liked his work and was unashamed.

  I should probably get another latte. I’ve just been sitting here with this empty cup. But then I’ll start to get jittery. I’ll get a decaf. No, that’s stupid, it feels stupid to pay for a decaf. I can’t justify that.

  John was always impatient on the weekends; he missed the formal structure of the business week. When he was younger he used to stay late after school on Fridays and come in early on Mondays, a pattern his mother referred to with equal parts admiration and disdain as “studying overtime.”

  Jesus, I’ve written another loser.

  Now, John spent his weekends doing yard work at the Tudor house Rebecca left him after their divorce. Rebecca, with her almond eyes—both in shape and in color—could never be his enemy.

  That barista keeps looking at me. She’ll probably ask me to leave if I don’t buy something. She’s kind of attractive. Not her hair—her hair seems stringy—but her face is nice. I should really buy something.

  Their divorce was remarkably amicable. In fact, John would often tell his parents, “Rebecca and I are better friends now than when we were married!” In fact, John looked forward to the days when he and Rebecca, with their new partners, would reminisce about their marriage, seeing it in a positive light, like two mature adults.

  Maybe I’ll just get a pumpkin-spice loaf. That way I can still sit here without going through a whole production of buying a coffee and giving my name and feeling like an asshole while it gets made.

  But if John were being honest, the house did get lonely on the weekends. Rebecca’s parents had been generous enough to leave John the house even though they had paid for it. John was still struggling to get his short-story writing—I mean, his painting—career off the ground, and Rebecca and her family had been more than supportive, even during the breakup."""

summary = generate_summary(ctx)
summary = extract_content(summary)
print(summary)

In [ ]:
#@title beautify the output

from IPython.display import Markdown as md
md(summary)